In [ ]:
!pip install requests


In [ ]:
import requests
import json
import pandas as pd
from pandas import json_normalize



In [ ]:
# Step 1: Generate Token 14 Days
url = "https://api.newsroom.bi/api/user/signin"
payload = {
    "email": "mbarber@infobae.com",
    "password": "IEd6lAur57@K#NlXFUeT"
}

response = requests.post(url, json=payload)

if response.status_code == 200:
    data = response.json()
    token = data.get('token')
    print("Token generated:", token)
else:
    print("Failed to generate token. Status code:", response.status_code)
    print("Response:", response.text)

Token generated: eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJ1c2VySWQiOjEyMzYzLCJlbWFpbCI6Im1iYXJiZXJAaW5mb2JhZS5jb20iLCJyb2xlIjoiIiwiYWNscyI6eyJpbnRlZ3JhdGlvbnM6Y29ubmVjdCI6MCwiaW5zaWdodHM6c2F2ZWQiOjAsImFjYWRlbXk6ZG9jdSI6MCwiY29tcGFzczptYXJmZWVsY2FzdCI6MCwiY29tcGFzczpwYWdpbmF0aW9uIjowLCJjb21wYXNzOmRyaWxsZG93biI6MCwiY29tcGFzczphZmZpbGlhdGlvbiI6MCwiYWJ0ZXN0Omhpc3RvcmljIjowLCJjb21wYW55OnVzZXJqb3VybmV5IjowLCJjb21wYXNzOnJlYWx0aW1lUmV2ZW51ZSI6MCwiY29tcGFzczpkYXRlcGlja2VyIjowLCJjb21wYXNzOmRldGFpbHMiOjAsImFjdGlvbnM6dmVjdG9yczMwZGF5cyI6MSwiY29tcGFzczpwcmVkaWN0aXZlIjowLCJodWQ6c2V0dGluZ3MiOjAsImluc2lnaHRzOmV4cGxvcmU6YW5ub3RhdGlvbnMiOjAsImNvbXBhbnk6b3JnYW5pemF0aW9uIjowLCJkYXNoYm9hcmRzOnBsYXlib29rcyI6MCwiaW5zaWdodHM6ZXhwbG9yZSI6MCwiY29tcGFzczpyZWFsdGltZVNlbGVjdE1ldHJpYyI6MCwidXNlcjppbXBlcnNvbmF0aW9uIjowLCJjb21wYXNzOnNzb1ZpZXciOjAsImluc2lnaHRzOnJlY2lyY3VsYXRpb24iOjAsImh1ZDpyZWNpcmN1bGF0aW9uIjowLCJodWQ6ZmlsdGVycyI6MCwiY29tcGFzczpwbG90MmdyYXBoIjowLCJ0cmVuZHM6bWFuYWdlIjowLCJjb3BpbG90OmhlbHBmdWxuZXNzIjowLC

In [ ]:
# Step 2: Get the first 10 urls with the most traffic during the last 7 days
url = "https://api.newsroom.bi/api/dashboard/query"
headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

payload = {
    "dates": {
        "last": {
            "number": 7,
            "dimension": "day"
        }
    },
    "granularity": "monthly",
    "filters": [
    ],
    "groupBy": [
        "url"

    ],
    "metrics": [
        "uniqueUsers"
    ],
    "order": {
        "metric": "uniqueUsers",
        "sort": "DESC"
    },

    "limit": 10,
    "from": 0
}

response = requests.post(url, headers=headers, json=payload)

if response.status_code == 200:
    json_data = response.json()
    print("Query response:", json.dumps(json_data, indent=4))
else:
    print("Failed to execute query. Status code:", response.status_code)
    print("Response:", response.text)
    # Extract the 'values' part of the JSON
values = json_data[0]['actualData']['values']

# Normalize the JSON data
normalized_data = pd.json_normalize(
    values,
    record_path=['items'],
    meta=['total'],
    record_prefix='item_'
)

# Filter the DataFrame to only include 'url' types
df = normalized_data[normalized_data['item_type'] == 'url'][['item_value', 'total']]
df.columns = ['URL', 'Unique Users']

# Display the DataFrame
print(df)



Query response: [
    {
        "metric": "uniqueUsers",
        "total": 41453862,
        "granularity": "monthly",
        "availableGranularities": [
            "hourly",
            "daily",
            "weekly"
        ],
        "actualData": {
            "total": 41453862,
            "values": [
                {
                    "key": "de148314ff25c4b856c27196e7d40379",
                    "total": 5678852,
                    "items": [
                        {
                            "id": "https://experiences.mrf.io/recommenderexperience/",
                            "value": "https://experiences.mrf.io/recommenderexperience/",
                            "type": "url"
                        }
                    ]
                },
                {
                    "key": "dd8e7fe51ba9869a4afbec0511eb1751",
                    "total": 2685962,
                    "items": [
                        {
                            "id": "https://www.infobae

In [ ]:
df

,URL,Unique Users
0,https://experiences.mrf.io/recommenderexperience/,5669890
1,https://www.infobae.com/,2863759
2,https://www.infobae.com/espana/2024/07/03/los-...,595830
3,https://www.infobae.com/espana/2024/07/05/desp...,590317
4,https://www.infobae.com/peru/2024/07/02/isabel...,566241
5,https://www.infobae.com/mexico/2024/07/04/como...,488721
6,https://www.infobae.com/espana/2024/07/01/una-...,460671
7,https://www.infobae.com/virales/2024/06/28/pus...,452626
8,https://www.infobae.com/mexico/2024/07/03/hura...,444448
9,https://www.infobae.com/mexico/2024/07/04/mari...,396038


In [ ]:
json_data

[{'metric': 'uniqueUsers',
  'total': 42795207,
  'granularity': 'monthly',
  'availableGranularities': ['hourly', 'daily', 'weekly'],
  'actualData': {'total': 42795207,
   'values': [{'key': 'de148314ff25c4b856c27196e7d40379',
     'total': 5669890,
     'items': [{'id': 'https://experiences.mrf.io/recommenderexperience/',
       'value': 'https://experiences.mrf.io/recommenderexperience/',
       'type': 'url'}]},
    {'key': 'dd8e7fe51ba9869a4afbec0511eb1751',
     'total': 2863759,
     'items': [{'id': 'https://www.infobae.com/',
       'value': 'https://www.infobae.com/',
       'type': 'url'}]},
    {'key': '4ca5181bb1bdfe41e16b923e11bd257c',
     'total': 595830,
     'items': [{'id': 'https://www.infobae.com/espana/2024/07/03/los-trabajadores-de-carrefour-denuncian-un-clima-laboral-toxico-la-empresa-coloca-carteles-avisando-del-numero-de-empleados-que-estan-de-baja/',
       'value': 'https://www.infobae.com/espana/2024/07/03/los-trabajadores-de-carrefour-denuncian-un-clima-l

In [ ]:
# Step 3: Query: multiples metricas (if needed)
payload = {
    "dates": {
        "last": {
            "number": 30,
            "dimension": "minute"
        }
    },
    "filters": [],
    "groupBy": [],
    "metrics": [
        "uniqueUsers",
        "anotherMetric"  # Add additional metrics as needed
    ],
    "order": {
        "metric": "uniqueUsers",
        "sort": "DESC"
    },
    "granularity": "realtime",
    "limit": 100,
    "from": 0
}

response = requests.post(url, headers=headers, json=payload)

if response.status_code == 200:
    data = response.json()
    print("Query response:", json.dumps(data, indent=4))
else:
    print("Failed to execute query. Status code:", response.status_code)
    print("Response:", response.text)






Query response: [
    {
        "metric": "uniqueUsers",
        "total": 338759,
        "granularity": "realtime",
        "availableGranularities": [
            "realtime",
            "hourly",
            "daily"
        ],
        "actualData": {
            "total": 338759,
            "values": [
                {
                    "key": "cfab1ba8c67c7c838db98d666f02a132",
                    "total": 338759,
                    "items": [
                        {
                            "id": "--",
                            "value": "--",
                            "type": "metric"
                        }
                    ]
                }
            ],
            "data": [
                {
                    "cfab1ba8c67c7c838db98d666f02a132": 111919,
                    "label": "2024-07-07 10:40"
                },
                {
                    "cfab1ba8c67c7c838db98d666f02a132": 111174,
                    "label": "2024-07-07 10:45"
        

In [ ]:
query_payload_page_views = {
    "dates": {
        "last": {
            "number": 60,
            "dimension": "minute"
        }
    },
   "filters": [
        {

        }],
    "groupBy": [],
    "metrics": [
        "pageViewsTotal"
    ],
    "order": {
         "metric": "uniqueUsers",
        "sort": "DESC"
   },
    "granularity": "hour",
    "limit": 100,
    "from": 0
}
response = requests.post(url, headers=headers, json=payload)

if response.status_code == 200:
    data = response.json()
    print("Query response:", json.dumps(data, indent=4))
else:
    print("Failed to execute query. Status code:", response.status_code)
    print("Response:", response.text)

Query response: [
    {
        "metric": "uniqueUsers",
        "total": 41453862,
        "granularity": "monthly",
        "availableGranularities": [
            "hourly",
            "daily",
            "weekly"
        ],
        "actualData": {
            "total": 41453862,
            "values": [
                {
                    "key": "de148314ff25c4b856c27196e7d40379",
                    "total": 5678852,
                    "items": [
                        {
                            "id": "https://experiences.mrf.io/recommenderexperience/",
                            "value": "https://experiences.mrf.io/recommenderexperience/",
                            "type": "url"
                        }
                    ]
                },
                {
                    "key": "dd8e7fe51ba9869a4afbec0511eb1751",
                    "total": 2685962,
                    "items": [
                        {
                            "id": "https://www.infobae

In [ ]:
query_google_views = {
    "dates": {
        "last": {
            "number": 1,
            "dimension": "day"
        }
    },
    "granularity": "daily",
    "filters": [{
        "filter": "source",
        "op": "eq",
        "value": ["Google Discover"]
    }],
    "groupBy": [
        "url"
    ],
    "metrics": [
        "uniqueUsers"
    ],
    "order": {
        "metric": "uniqueUsers",
        "sort": "DESC"
    },

    "limit": 100,
    "from": 0
}

response = requests.post(url, headers=headers, json=payload)

if response.status_code == 200:
    json_data = response.json()
    print("Query response:", json.dumps(json_data, indent=4))
else:
    print("Failed to execute query. Status code:", response.status_code)
    print("Response:", response.text)
    # Extract the 'values' part of the JSON
values = json_data[0]['actualData']['values']

# Normalize the JSON data
normalized_data = pd.json_normalize(
    values,
    record_path=['items'],
    meta=['total'],
    record_prefix='item_'
)

# Filter the DataFrame to only include 'url' types
df = normalized_data[normalized_data['item_type'] == 'url'][['item_value', 'total']]
df.columns = ['URL', 'Unique Users']

# Display the DataFrame
print(df)


Query response: [
    {
        "metric": "uniqueUsers",
        "total": 41453862,
        "granularity": "monthly",
        "availableGranularities": [
            "hourly",
            "daily",
            "weekly"
        ],
        "actualData": {
            "total": 41453862,
            "values": [
                {
                    "key": "de148314ff25c4b856c27196e7d40379",
                    "total": 5678852,
                    "items": [
                        {
                            "id": "https://experiences.mrf.io/recommenderexperience/",
                            "value": "https://experiences.mrf.io/recommenderexperience/",
                            "type": "url"
                        }
                    ]
                },
                {
                    "key": "dd8e7fe51ba9869a4afbec0511eb1751",
                    "total": 2685962,
                    "items": [
                        {
                            "id": "https://www.infobae

In [ ]:
 #query_Visitor_loyalty
query_google_views = {
     "dates": {
        "last": {
            "number": 1,
            "dimension": "day"
        }
    },
    "granularity": "daily",
    "filters": [],
    "groupBy": [
        "visitorFrequency"
    ],
    "metrics": [
        "uniqueUsers"
    ],
    "order": {
        "metric": "uniqueUsers",
        "sort": "DESC"
    },

    "limit": 100,
    "from": 0
   }
response = requests.post(url, headers=headers, json=payload)

if response.status_code == 200:
    data = response.json()
    print("Query response:", json.dumps(data, indent=4))
else:
    print("Failed to execute query. Status code:", response.status_code)
    print("Response:", response.text)

Query response: [
    {
        "metric": "uniqueUsers",
        "total": 431238,
        "granularity": "realtime",
        "availableGranularities": [
            "realtime",
            "hourly",
            "daily"
        ],
        "actualData": {
            "total": 431238,
            "values": [
                {
                    "key": "cfab1ba8c67c7c838db98d666f02a132",
                    "total": 431238,
                    "items": [
                        {
                            "id": "--",
                            "value": "--",
                            "type": "metric"
                        }
                    ]
                }
            ],
            "data": [
                {
                    "cfab1ba8c67c7c838db98d666f02a132": 164603,
                    "label": "2024-07-02 14:45"
                },
                {
                    "cfab1ba8c67c7c838db98d666f02a132": 163833,
                    "label": "2024-07-02 14:50"
        

In [ ]:
query_google_views = {
    "dates": {
        "last": {
            "number": 1,
            "dimension": "day"
        }
    },
    "granularity": "daily",
    "filters": [{
        "filter": "source",
        "op": "eq",
        "value": ["Google Discover"]
    }],
    "groupBy": [
        "url"
    ],
    "metrics": [
        "uniqueUsers"
    ],
    "order": {
        "metric": "uniqueUsers",
        "sort": "DESC"
    },

    "limit": 100,
    "from": 0
}
response = requests.post(url, headers=headers, json=payload)

if response.status_code == 200:
    data = response.json()
    print("Query response:", json.dumps(data, indent=4))
else:
    print("Failed to execute query. Status code:", response.status_code)
    print("Response:", response.text)

Query response: [
    {
        "metric": "uniqueUsers",
        "total": 434105,
        "granularity": "realtime",
        "availableGranularities": [
            "realtime",
            "hourly",
            "daily"
        ],
        "actualData": {
            "total": 434105,
            "values": [
                {
                    "key": "cfab1ba8c67c7c838db98d666f02a132",
                    "total": 434105,
                    "items": [
                        {
                            "id": "--",
                            "value": "--",
                            "type": "metric"
                        }
                    ]
                }
            ],
            "data": [
                {
                    "cfab1ba8c67c7c838db98d666f02a132": 164603,
                    "label": "2024-07-02 14:45"
                },
                {
                    "cfab1ba8c67c7c838db98d666f02a132": 163833,
                    "label": "2024-07-02 14:50"
        

In [ ]:
query_payload_top_articles = {
    "dates": {
        "last": {
            "number": 24,
            "dimension": "hour"
        }
    },
    "filters": [],
    "groupBy": [
        "articleId"
    ],
    "metrics": [
        "engagement"
    ],
    "order": {
        "metric": "engagement",
        "sort": "DESC"
    },
    "granularity": "day",
    "limit": 10,
    "from": 0
}
response = requests.post(url, headers=headers, json=payload)

if response.status_code == 200:
    data = response.json()
    print("Query response:", json.dumps(data, indent=4))
else:
    print("Failed to execute query. Status code:", response.status_code)
    print("Response:", response.text)

Query response: [
    {
        "metric": "uniqueUsers",
        "total": 434105,
        "granularity": "realtime",
        "availableGranularities": [
            "realtime",
            "hourly",
            "daily"
        ],
        "actualData": {
            "total": 434105,
            "values": [
                {
                    "key": "cfab1ba8c67c7c838db98d666f02a132",
                    "total": 434105,
                    "items": [
                        {
                            "id": "--",
                            "value": "--",
                            "type": "metric"
                        }
                    ]
                }
            ],
            "data": [
                {
                    "cfab1ba8c67c7c838db98d666f02a132": 164603,
                    "label": "2024-07-02 14:45"
                },
                {
                    "cfab1ba8c67c7c838db98d666f02a132": 163833,
                    "label": "2024-07-02 14:50"
        

In [ ]:
query_social_traffic_articles = {
    "dates":{
        "last":{
            "number":"7",
            "dimension":"day"
        }
    },
    "filters":[{
        "filter":"mediumInternal",
        "op":"eq",
        "value":["social"]
    }],
    "groupBy":[
        "url"
    ],
    "metrics":[
        "uniqueUsers"
    ],
    "order":{
        "metric":"uniqueUsers",
        "sort":"DESC"
    },
    "granularity":"daily"
}
response = requests.post(url, headers=headers, json=payload)

if response.status_code == 200:
    data = response.json()
    print("Query response:", json.dumps(data, indent=4))
else:
    print("Failed to execute query. Status code:", response.status_code)
    print("Response:", response.text)

Query response: [
    {
        "metric": "uniqueUsers",
        "total": 434105,
        "granularity": "realtime",
        "availableGranularities": [
            "realtime",
            "hourly",
            "daily"
        ],
        "actualData": {
            "total": 434105,
            "values": [
                {
                    "key": "cfab1ba8c67c7c838db98d666f02a132",
                    "total": 434105,
                    "items": [
                        {
                            "id": "--",
                            "value": "--",
                            "type": "metric"
                        }
                    ]
                }
            ],
            "data": [
                {
                    "cfab1ba8c67c7c838db98d666f02a132": 164603,
                    "label": "2024-07-02 14:45"
                },
                {
                    "cfab1ba8c67c7c838db98d666f02a132": 163833,
                    "label": "2024-07-02 14:50"
        